In [ ]:
!pip install degirum

In [ ]:
import cv2
import torch
import requests
import numpy as np
import degirum as dg
import torch.nn.functional as F

In [ ]:
# Connect to DeGirum AI inference engine
zoo = dg.connect("@cloud", "https://cs.degirum.com/degirum/kvk_upload_test", "<my cloud API access token>")
model = zoo.load_model('clip--224x224_float_openvino_cpu_4', input_image_format = "RAW")

In [ ]:
prompts = ["People Running", "People Talking", "People Fighting", "People Laughing", "People Dancing"]

# DeGirum API endpoint URL for computing embeddings
api_url = 'http://100.97.6.89:8585/compute_embeddings'
data = {'text_prompts': prompts}
response = requests.post(api_url, json=data)
embeddings = response.json()["embeddings"]

In [ ]:
!wget https://medium.com/r/?url=https%3A%2F%2Fpixabay.com%2Fphotos%2Frace-marathon-runners-athletes-932254%2F

In [ ]:
import cv2
image = cv2.imread("race-932254_640.jpg")
dg_cloud_output = model(image).results[0]["data"]
dg_cloud_output_reshaped = dg_cloud_output.reshape(-1)

In [ ]:
# Compute the cosine similarity between two vectors.
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
similarities = [cosine_similarity(dg_cloud_output_reshaped, np.array(embedding).reshape(-1)) for embedding in embeddings]
similarities_tensor = torch.tensor(similarities, dtype=torch.float32)
softmax_scores = F.softmax(similarities_tensor, dim=0)
max_index = torch.argmax(softmax_scores).item()

for index, prompt in enumerate(prompts):
    print(f"{prompt} - {softmax_scores[index]*100:.2f}%")